In [1]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx

In [2]:
class State(enum.IntEnum):
    SUSCEPTIBLE = 0
    VACCINATED = 1
    INFECTED_S = 2
    INFECTED_V = 3
    RECOVERED_S = 4
    RECOVERED_V = 5

In [3]:
class MyAgent(Agent):
    """ An agent in an epidemic model."""
    def __init__(self, SV,unique_id, model):
        super().__init__(unique_id, model)
        self.SV = SV
        if self.SV == "S":
            self.state = State.SUSCEPTIBLE
        else:
            self.state = State.VACCINATED 


    def move(self):
        """Move the agent"""

        possible_steps = [
            node
            for node in self.model.grid.get_neighbors(self.pos, include_center=False)
            if self.model.grid.is_cell_empty(node)
        ]
        if len(possible_steps) > 0:
            new_position = self.random.choice(possible_steps)                 
            self.model.grid.move_agent(self, new_position)
    
    def status(self):
        """Check infection status"""
        
        if self.state == State.INFECTED_S: 
            recovery_rate_Is = 1 - np.exp(-self.model.gamma)
            
            recovered = np.random.choice([0,1], p=[recovery_rate_Is,1-recovery_rate_Is])
            if recovered == 0:
                self.state = State.RECOVERED_S

                
        if self.state == State.INFECTED_V: 
            recovery_rate_Iv = 1 - np.exp(-self.model.gamma * self.model.delta)
            
            recovered = np.random.choice([0,1], p=[recovery_rate_Iv,1-recovery_rate_Iv])
            if recovered == 0:
                self.state = State.RECOVERED_V




    def contact(self):
        """Find close contacts and infect"""
        
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_vaccinated_neighbors = [
            agent
            for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            if ((agent.state is State.SUSCEPTIBLE) or (agent.state is State.VACCINATED))
        ]
        
        for other in susceptible_vaccinated_neighbors:        
            if self.random.random() > self.model.beta:
                continue

                #If Contatct Wtih Other agent
            if self.state is State.INFECTED_S and other.state is State.SUSCEPTIBLE:
                rate = 1 - np.exp(-self.model.beta)
                decision = np.random.choice([0,1], p=[rate,1-rate])

                if decision == 0:
                    other.state = State.INFECTED_S


            if self.state is State.INFECTED_S and other.state is State.VACCINATED:   
                r = self.model.beta * (1-self.model.eta)-self.model.beta * (1-self.model.eta)*self.model.effectiveness
                rate = 1 - np.exp(-r)
                decision = np.random.choice([0,1], p=[rate,1-rate])

                if decision == 0:
                    other.state = State.INFECTED_V


            if self.state is State.INFECTED_V and other.state is State.SUSCEPTIBLE:
                rate = 1- np.exp(-self.model.beta)
                decision = np.random.choice([0,1], p=[rate,1-rate])

                if decision == 0:
                    other.state = State.INFECTED_S


            if self.state is State.INFECTED_S and other.state is State.VACCINATED:
                r  = self.model.beta * (1-self.model.eta)-self.model.beta * (1-self.model.eta)*self.model.effectiveness
                rate = 1- np.exp(-r)
                decision = np.random.choice([0,1], p=[rate,1-rate])

                if decision == 0:
                    other.state = State.INFECTED_V

    def step(self):
        self.status()
        self.move()
        self.contact()
        
    def toJSON(self):        
        d = self.unique_id
        return json.dumps(d, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

In [ ]:
class SIRVModel(Model):
    """A model for infection spread."""
    
    def __init__(self, N=10,num_vaccination = 5,avg_node_deg=10,
                beta = 0.833,gamma = 1/3, delta = 3,eta = 0.3, effectiveness = 0.5):
        
        self.num_agents = N
        self.num_vaccination = num_vaccination
        
        
        #parameters
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.eta = eta
        self.effectiveness = effectiveness
        

        
        self.connection_prob = avg_node_deg/self.num_agents
        self.G = nx.erdos_renyi_graph(n=self.num_agents, p=self.connection_prob)
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)

        
        self.running = True
        
        
        
        # Agent Creation 
        for i, node in enumerate(self.G.nodes()):
            if i < self.num_vaccination:
                a = MyAgent("V",i+1, self)
            else:
                a = MyAgent("S", i+1, self)
            self.schedule.add(a)
            self.grid.place_agent(a,node)
            

            # If want to make some agent initially infected but here we take initially 
            if a.state == State.VACCINATED:
                infected = np.random.choice([0,1], p=[0.005,0.995])
                
                if infected == 0:
                    a.state = State.INFECTED_V

                
            elif a.state == State.SUSCEPTIBLE:
                infected = np.random.choice([0,1], p=[0.015,0.985])
                if infected == 0:
                    a.state = State.INFECTED_S

                
        # Collecting States of agent
        self.datacollector = DataCollector(
            #model_reporters={"Gini": compute_gini}, 
            agent_reporters={"State": "state"})
            
        
 
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()